In [1]:
import json
import psycopg2
import re
import os
from psycopg2.extras import execute_batch

conn = psycopg2.connect(
    dbname="fpkg0_1",
    user='postgres',
    password='postgres',
    host="localhost",
    port="5431"
)

cur = conn.cursor()

cur.execute("LOAD 'age';")
cur.execute("SET search_path = ag_catalog, \"$user\", public;")

conn.commit()

OperationalError: connection to server at "localhost" (::1), port 5431 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5431 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [4]:
query = f"""
SELECT * FROM cypher('fcsv', $$ 
    MATCH (d) -[e:HasContraindication]-> (c)
    RETURN e
$$) AS (result agtype);
"""

try:
    cur.execute(query)
    results = cur.fetchall()
    print(len(results))
    for res in results[2000:2020]:
        res = re.sub(r'::\w+$', '', res[0])
        print(res)
except Exception as e:
    print(e)
    conn.rollback()

226623
{"id": 5348024557516217, "label": "HasContraindication", "end_id": 2533274790476847, "start_id": 844424930132788, "properties": {"level": "CONTRE-INDICATION ABSOLUE", "comment": "", "subtypes": "HYPERSENSIBILITE GALACTOSE (INTOLERANCE),HYPERSENSIBILITE GLUCOSE (INTOLERANCE),HYPERSENSIBILITE LACTOSE (INTOLERANCE)"}}
{"id": 5348024557526919, "label": "HasContraindication", "end_id": 2533274790476847, "start_id": 844424930133450, "properties": {"level": "CONTRE-INDICATION ABSOLUE", "comment": "", "subtypes": "HYPERSENSIBILITE GALACTOSE (INTOLERANCE),HYPERSENSIBILITE GLUCOSE (INTOLERANCE),HYPERSENSIBILITE LACTOSE (INTOLERANCE)"}}
{"id": 5348024557521904, "label": "HasContraindication", "end_id": 2533274790476847, "start_id": 844424930133109, "properties": {"level": "CONTRE-INDICATION ABSOLUE", "comment": "", "subtypes": "HYPERSENSIBILITE GALACTOSE (INTOLERANCE),HYPERSENSIBILITE GLUCOSE (INTOLERANCE),HYPERSENSIBILITE LACTOSE (INTOLERANCE)"}}
{"id": 5348024557521899, "label": "HasCont

In [37]:
conn.rollback()